In [51]:
from tensorflow.keras.datasets.mnist import load_data
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [16]:
(x_train, y_train), (x_test, y_test) = load_data()

In [26]:
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)
y_train = OneHotEncoder().fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = OneHotEncoder().fit_transform(y_test.reshape(-1, 1)).toarray()

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('shape of x_train:', x_train.shape)
print('shape of y_train:', y_train.shape)
print('shape of x_val:', x_val.shape)
print('shape of y_val:', y_val.shape)
print('shape of x_test:', x_test.shape)
print('shape of y_test:', y_test.shape)

shape of x_train: (48000, 784)
shape of y_train: (48000, 10)
shape of x_val: (12000, 784)
shape of y_val: (12000, 10)
shape of x_test: (10000, 784)
shape of y_test: (10000, 10)


### 将mnist数据集读取出来，并保存为图片文件

In [45]:
from PIL import Image
import os

In [47]:
save_dir = 'E:\\DL_datasets\\mnist\\raw'
if not os.path.exists(save_dir):
    os.makedir(save_dir)
    
# 保存前n张图片
n = 20
for i in range(n):
    image_array = x_train[i, :]
    image_array = image_array.reshape(28, 28)
    filename = os.path.join(save_dir, 'mnist_train_{}.jpg'.format(i))
    im = Image.fromarray(image_array)
    im.save(filename)

### 使用softmax模型进行分类

In [66]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 模型的输出
y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_  * tf.log(y)))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

batch_size = 128
for i in range(48000//batch_size):
    batch_xs = x_train[i*batch_size: (i+1)*batch_size]
    batch_ys = y_train[i*batch_size: (i+1)*batch_size]
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})
    if i % 100 == 0:
        print(i)

0
100
200
300


In [65]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: x_test, y_: y_test}))

0.098
